# ADA - Banco de Dados I - Projeto Final

## Professor: Tiago Dias

## Grupo 1:

* Alana Nunes
* Deivid Gabriel
* Ewerton Costa
* Hugo Negrão
* Vinicius Silva

### TEMA: Consumo de API pelo python com integração a um Banco de Dados


**Contexto**

A API de Star Wars, ou "swapi" (Swah-pee) é a primeira fonte de dados quantificada e acessível por meio de programação para todos os dados do universo canônico de Star Wars!

esses dados foram obtidos de https://swapi.dev/ e são apenas para fins educativos.


Está API contém os Dados de Star Wars: Planetas, Naves Espaciais, Veículos, Pessoas, Filmes e Espécies de todos os SETE filmes de Star Wars.


**Proposta**

Este projeto tem por finalidade o consumo de uma API através do python e a construção de um Bando de Dados com os conhecimentos obtidos no módulo DS-PY-006 BANCO DE DADOS I - Turma 889 - Diversidade Tech - Suzano.

In [1]:
# Bibliotecas Utilizadas
import numpy as np
import pandas as pd
import requests
import psycopg2
import time

### Conexão do Python com a API

In [2]:
def get_dados_semi_completos_in_df(tipo: str) -> pd.DataFrame:
    '''
    Extrai dados via The Star Wars API, limitado a 10 linhas.

    Parâmetros:
        tipo (str): tipo de dado a ser extraído - planets, spaceships, people, vehicles, films ou species

    Retorna:
        df (dataframe): dado extraído em formato de pandas dataframe
    '''
    url = f'https://swapi.dev/api/{tipo}/'
    resposta = requests.get(url)
    dados = resposta.json()
    print(resposta.status_code)
    df = pd.DataFrame.from_dict(dados.get('results'))
    return df


def get_dados_completos_in_df(tipo):
    '''
    Extrai dados via The Star Wars API, pegando todos os registros.
    
    Parâmetros:
        tipo (str): tipo de dado a ser extraído - planets, starships, people, vehicles, films ou species
        
    Retorna:
        df (dataframe): dado extraído em formato de pandas dataframe
    '''
    df = pd.DataFrame()
    
    for i in range(1,91):
        try:
            url = f'https://swapi.dev/api/{tipo}/{i}/'
            resposta = requests.get(url)
            dados = resposta.json()

            if resposta.status_code == 200:
                df1 = pd.DataFrame.from_dict(dados, orient ='index')
                df1 = df1.transpose()
                df1[f'id_{tipo}'] = i
#                     df = df.append(df1)
                df = pd.concat([df, df1])
                time.sleep(1)
            else:
                print(i, ' - ', resposta.status_code)
                time.sleep(1)
                pass
        except:
            print(f'{i} - Erro na requisição')
            time.sleep(1)
    return df


def tratar_df(df: pd.DataFrame, tipo) -> pd.DataFrame:
    '''
    Trata o dataframe para manipulação.

    Parâmetros:
        df (dataframe): dataframe a ser tratado
        tipo (str): tipo de dado do dataframe - planets, starships, people

    Retorna:
        df (dataframe): dataframe tratado
    '''
    df = df.replace('unknown', np.nan)
    df = df.replace('n/a', np.nan)
    df = df.replace('N/A', np.nan)
    df = df.fillna(0)
    return df

def selecionar_colunas(df: pd.DataFrame, tipo: str) -> pd.DataFrame:
    '''
    Seleciona apenas as colunas que serão usadas para popular o banco de dados

    Parâmetros:
        df (dataframe): dataframe a ser tratado
        tipo (str): tipo de dado do dataframe - planets, starships, people

    Retorna:
        df (dataframe): dataframe tratado
    '''
    if tipo == 'planets':
        new_cols = ['id_planets', 'name', 'diameter', 'climate', 'gravity', 'terrain', 'population']
        df = df[new_cols]
        return df
        
    elif tipo == 'people':
        new_cols = ['id_people', 'name', 'height', 'mass', 'hair_color', 'skin_color', 'birth_year', 'gender', 'homeworld']
        df = df[new_cols]
        return df

    elif tipo == 'starships':
        new_cols = ['id_starships', 'name', 'model', 'manufacturer', 'cost_in_credits', 'length', 'crew', 'passengers', 'consumables', 'pilots']
        df = df[new_cols]
        return df

def save_csv(df: pd.DataFrame, tipo: str):
    '''
    Salva dataframe em formato csv.
    '''
    df.to_csv(f'{tipo}.csv', sep=';', encoding='utf-8', index=False)

def tratar_coluna_chave_estrangeira(row, coluna: str):
    '''
    Tratar coluna que vem em formato de url para o formato de id. A função será utilizada com o método apply.
    
    Parâmetros:
        coluna (str): coluna a ser tratada

    Retorna:
        row[coluna] (series): coluna tratada
    '''
    if coluna == 'homeworld':
        row[coluna] = row[coluna][-3:-1]
        if '/' in row[coluna]:
            row[coluna] = row[coluna][-1]
        return row[coluna]

    elif coluna == 'pilots':
        if row[coluna] == '[]':
            row[coluna] = np.nan
        else:
            row[coluna] = row[coluna][31:33]
            if '/' in row[coluna]:
                row[coluna] = row[coluna][-2]
        return row[coluna]
    
def adicionar_linha_nula_id0(df: pd.DataFrame, tipo: str) -> pd.DataFrame:
    '''
    Adiciona uma linha nula com id_people = 0 se dataframe people ou id_planets = 0 se dataframe planets

    Parâmetros:
        df (dataframe): dataframe a ser tratado
        tipo (str): tipo de dado do dataframe - planets, starships, people

    Retorna:
        df (dataframe): dataframe tratado
    '''
    if tipo == 'people':
        dict_people_id0 = {
            'id_people': 0,
            'name': np.nan,
            'height': 0,
            'mass': 0,
            'hair_color': np.nan, 
            'skin_color': np.nan,
            'birth_year':np.nan, 
            'gender':np.nan, 
            'homeworld': 0
        }
        df = df.append(dict_people_id0, ignore_index=True)
        return df
    
    elif tipo == 'planets':
        dict_planets_id0 = {
            'id_planets': 0,
            'name': np.nan,
            'diameter': 0,
            'climate': np.nan,
            'gravity': np.nan, 
            'terrain': np.nan,
            'population':0
        }
        df = df.append(dict_planets_id0, ignore_index=True)
        return df

### Aquisição dos dados

In [3]:
%%time
# Para recuperar os dados semi_completos (esse link só retorna 10 linhas), no exemplo, para dados de people (pessoas)
df = get_dados_semi_completos_in_df(tipo = 'people')
df.head()

200
CPU times: total: 15.6 ms
Wall time: 8.96 s


,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/


In [3]:
# Recuperar os dados completos de people (pessoas)
df_people = get_dados_completos_in_df(tipo = 'people')
# Recuperar os dados completos de starships (naves)
df_starships = get_dados_completos_in_df(tipo = 'starships')
# Recuperar os dados completos de planets (planetas)
df_planets = get_dados_completos_in_df(tipo = 'planets')

61  -  404
62  -  404
63  -  404
64  -  404
65  -  404
66  -  404
67  -  404
68  -  404
69  -  404
70  -  404
71  -  404
72  -  404
73  -  404
74  -  404
75  -  404
76  -  404
77  -  404
78  -  404
79  -  404
80  -  404
81  -  404
82  -  404
83  -  404
84  -  404
85  -  404
86  -  404
87  -  404
88  -  404
89  -  404
90  -  404


In [5]:
# Verificar as informações do dataframe
df_planets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 0
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             60 non-null     object
 1   rotation_period  60 non-null     object
 2   orbital_period   60 non-null     object
 3   diameter         60 non-null     object
 4   climate          60 non-null     object
 5   gravity          60 non-null     object
 6   terrain          60 non-null     object
 7   surface_water    60 non-null     object
 8   population       60 non-null     object
 9   residents        60 non-null     object
 10  films            60 non-null     object
 11  created          60 non-null     object
 12  edited           60 non-null     object
 13  url              60 non-null     object
dtypes: object(14)
memory usage: 7.0+ KB


In [6]:
# Verificar as informações do dataframe
df_people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 0
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        82 non-null     object
 1   height      82 non-null     object
 2   mass        82 non-null     object
 3   hair_color  82 non-null     object
 4   skin_color  82 non-null     object
 5   eye_color   82 non-null     object
 6   birth_year  82 non-null     object
 7   gender      82 non-null     object
 8   homeworld   82 non-null     object
 9   films       82 non-null     object
 10  species     82 non-null     object
 11  vehicles    82 non-null     object
 12  starships   82 non-null     object
 13  created     82 non-null     object
 14  edited      82 non-null     object
 15  url         82 non-null     object
dtypes: object(16)
memory usage: 10.9+ KB


In [7]:
# Verificar as informações do dataframe
df_starships.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 0
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    36 non-null     object
 1   model                   36 non-null     object
 2   manufacturer            36 non-null     object
 3   cost_in_credits         36 non-null     object
 4   length                  36 non-null     object
 5   max_atmosphering_speed  36 non-null     object
 6   crew                    36 non-null     object
 7   passengers              36 non-null     object
 8   cargo_capacity          36 non-null     object
 9   consumables             36 non-null     object
 10  hyperdrive_rating       36 non-null     object
 11  MGLT                    36 non-null     object
 12  starship_class          36 non-null     object
 13  pilots                  36 non-null     object
 14  films                   36 non-null     object
 15  created  

### Tratando os dataframes

In [5]:
# Limpando os null e os missings e os 'n/a'
df_starships = tratar_df(df = df_starships, tipo = 'starships')
df_people = tratar_df(df = df_people, tipo = 'people')
df_planets = tratar_df(df = df_planets,  tipo = 'planets')

In [6]:
#Selecionando as colunas válidas para o banco de dados
df_planets = selecionar_colunas(df = df_planets, tipo = 'planets')
df_people = selecionar_colunas(df = df_people, tipo = 'people')
df_starships = selecionar_colunas(df = df_starships, tipo = 'starships')

In [7]:
# Tratando colunas que vão servir como chave estrangeira
df_people['homeworld'] = df_people.apply(lambda row: tratar_coluna_chave_estrangeira(row, 'homeworld'), axis=1)
df_starships['pilots'] = df_starships.apply(lambda row: tratar_coluna_chave_estrangeira(row, 'pilots'), axis=1)

In [8]:
# Adicionando linhas com id0 para a tabela de people e planets, pois a coluna 'pilots'(correspondente a id_pessoa na tabela people) da tabela starships vai apresentar id com valores id = 0.
# E uma pessoa com id = 0, que não existe, vai precisar habitar em um planeta, por isso, também cria-se um planeta com id_planets = 0, inexistente.
df_people = adicionar_linha_nula_id0(df = df_people, tipo = 'people')
df_planets = adicionar_linha_nula_id0(df = df_planets, tipo = 'planets')

In [27]:
# Tratando dados problemáticos
df_people['mass'].loc[df_people['mass'] == '1,358'] = 1358
df_starships['passengers'].loc[df_starships['passengers'] == '843,342'] = 843342
df_starships['length'].loc[df_starships['length'] == '1,600'] = 1600
df_starships['crew'].loc[df_starships['crew'] == '30-165'] = 165
df_starships['crew'].loc[df_starships['crew'] == '47,060'] = 47060
df_starships['crew'].loc[df_starships['crew'] == '342,953'] = 342953
df_starships['crew'].loc[df_starships['crew'] == '279,144'] = 279144

In [8]:
# Salvar os dataframes em csv
save_csv(df = df_people, tipo = 'people')
save_csv(df = df_starships, tipo = 'starships')
save_csv(df = df_planets, tipo = 'planets')

## Conexão com o BD

In [41]:
def conexao_db():
    '''
    Conecta o Python ao banco de dados.
    '''
    conexao = psycopg2.connect(host='localhost',
                               database='db_star_wars',
                               user='postgres',
                               password='admin')
    return conexao

def executar_sql(sql: str):
    '''
    Executa query SQL.
    '''
    conexao = conexao_db()
    cursor = conexao.cursor()
    cursor.execute(sql)
    conexao.commit()
    conexao.close()

def executar2_sql(sql: str, values: list):
    '''
    Executa query SQL.
    '''
    conexao = conexao_db()
    cursor = conexao.cursor()
    cursor.execute(sql, values)
    conexao.commit()
    conexao.close()
    
def executar_sql_para_select(sql: str):
    '''
    Executa query SQL para SELECT.
    '''
    conexao = conexao_db()
    cursor = conexao.cursor()
    cursor.execute(query)
    resposta = cursor.fetchall() 
    return resposta

In [3]:
# Ler os arquivos csv
df_starships = pd.read_csv('starships.csv', sep = ';')
df_people = pd.read_csv('people.csv', sep = ';')
df_planets = pd.read_csv('planets.csv', sep = ';')

In [35]:
df_planets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_planets  60 non-null     int64 
 1   name        60 non-null     object
 2   diameter    60 non-null     int64 
 3   climate     60 non-null     object
 4   gravity     60 non-null     object
 5   terrain     60 non-null     object
 6   population  60 non-null     int64 
dtypes: int64(3), object(4)
memory usage: 3.4+ KB


In [10]:
# Cria tabela planeta
executar_sql("""CREATE TABLE IF NOT EXISTS planeta (
        id_planeta SERIAL NOT NULL PRIMARY KEY, 
        nome VARCHAR,
        diametro INT,
        clima VARCHAR,
        gravidade VARCHAR,
        terreno VARCHAR,
        populacao BIGINT);""")

In [63]:
df_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        82 non-null     object 
 1   height      81 non-null     float64
 2   mass        59 non-null     object 
 3   hair_color  77 non-null     object 
 4   skin_color  81 non-null     object 
 5   birth_year  43 non-null     object 
 6   gender      79 non-null     object 
 7   homeworld   82 non-null     object 
dtypes: float64(1), object(7)
memory usage: 5.2+ KB


In [12]:
# Cria tabela pessoa
executar_sql("""CREATE TABLE IF NOT EXISTS pessoa (
        id_pessoa SERIAL NOT NULL PRIMARY KEY, 
        nome VARCHAR NOT NULL,
        altura INT,
        massa INT,
        cor_cabelo VARCHAR,
        cor_pele VARCHAR,
        ano_nascimento VARCHAR,
        genero VARCHAR,
        id_planeta INTEGER NOT NULL,
        FOREIGN KEY (id_planeta) REFERENCES planeta(id_planeta));""")

In [64]:
df_starships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             36 non-null     object 
 1   model            36 non-null     object 
 2   manufacturer     36 non-null     object 
 3   cost_in_credits  26 non-null     float64
 4   length           36 non-null     object 
 5   crew             35 non-null     object 
 6   passengers       32 non-null     object 
 7   consumables      31 non-null     object 
 8   pilots           36 non-null     object 
dtypes: float64(1), object(8)
memory usage: 2.7+ KB


In [14]:
# Cria tabela naves
executar_sql("""CREATE TABLE IF NOT EXISTS naves (
        id_naves SERIAL NOT NULL PRIMARY KEY, 
        nome VARCHAR NOT NULL,
        modelo VARCHAR NOT NULL,
        fabricante VARCHAR NOT NULL,
        custo BIGINT,
        comprimento VARCHAR,
        qtd_tripulantes VARCHAR,
        qtd_passageiros VARCHAR,
        suprimento VARCHAR,
        id_pessoa INT,
        FOREIGN KEY (id_pessoa) REFERENCES pessoa(id_pessoa));""")

In [11]:
# Adiciona na tabela de planeta
for indice, linha in df_planets.iterrows():
    executar2_sql(f"""INSERT INTO planeta (id_planeta, nome, diametro, clima, gravidade, terreno, populacao) 
                VALUES (%s, %s, %s, %s, %s, %s, %s);""", linha[df_planets.columns.values])

In [13]:
# Adiciona na tabela de pessoa
for indice, linha in df_people.iterrows():
    executar2_sql(f"""INSERT INTO pessoa (id_pessoa, nome, altura, massa, cor_cabelo, cor_pele, ano_nascimento, genero, id_planeta) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);""", linha[df_people.columns.values])

In [15]:
# Adiciona na tabela de naves
for indice, linha in df_starships.iterrows():
    executar2_sql(f"""INSERT INTO naves (id_naves, nome, modelo, fabricante, custo, comprimento, qtd_tripulantes, qtd_passageiros, suprimento, id_pessoa) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", linha[df_starships.columns.values])

## Consultando dados no Postgres

In [46]:
# Quem são os 5 personagens mais altos?
query = "select id_pessoa, nome, altura from pessoa order by altura desc limit 5;"
resposta = executar_sql_para_select(query)
conexao.close()

resposta

[(57, 'Yarael Poof', 264),
 (80, 'Tarfful', 234),
 (72, 'Lama Su', 229),
 (13, 'Chewbacca', 228),
 (37, 'Roos Tarpals', 224)]